In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.patches as patches
import jax.numpy as jnp # this is a thin wrapper to NumPy within JAX
from jax import grad, hessian

mpl.rcParams['lines.linewidth'] = 2
mpl.rc('xtick', labelsize=18) 
mpl.rc('ytick', labelsize=18) 
mpl.rc('axes', labelsize=18) 
mpl.rc('font', size=18) 
plt.rcParams.update({
    'text.usetex': True,
    'font.family': 'serif',
})

# define objective function and constraints

In [2]:
f = lambda x: (3*x[...,0]**2 + x[...,1]**2 + 2*x[...,0]*x[...,1] + x[...,0] + 6*x[...,1]).squeeze()
c_1 = lambda x: (2.*x[...,0] + 3*x[...,1] - 4.).squeeze()
c_2 = lambda x: (x[...,0]).squeeze()
c_3 = lambda x: (x[...,1]).squeeze()
c = [c_1, c_2, c_3]
d = np.array([-4., 0., 0.]) # RHS of constraint equations

In [3]:
m = len(c)
n = 2
xk = np.array([[3., 2.]]) # starting point
Bk = hessian(f)(xk).squeeze() # function hessian
Ck = np.concatenate([grad(c[i])(xk) for i in range(m)])

In [4]:
Wk = []
# find which constraints are active
for i in range(m):
    print(f'constraint {i+1} at {xk} {c[i](xk)}')
    if np.abs(c[i](xk)) <= 1e-12:
        Wk.append(i)
print(f'{len(Wk)} of {m} constraints are active')

constraint 1 at [[3. 2.]] 8.0
constraint 2 at [[3. 2.]] 3.0
constraint 3 at [[3. 2.]] 2.0
0 of 3 constraints are active


# Now we construct the KKT system $L [x, \lambda]^T = y$

In [5]:
# Construct KKT system
n_active = len(Wk) 
lk_I = np.zeros(m) # Lagrange multipliers for all inequality constraints
Z = np.zeros([n_active, n_active])

if n_active > 0:
    L = np.block([[Bk, -Ck[Wk].T],
                  [-Ck[Wk], Z]])
else:
    L = Bk # no active constraints

y    = np.append(-grad(f)(xk).squeeze() + np.matmul(Ck[Wk].T, lk_I[Wk]), np.zeros(n_active) )
soln = np.linalg.solve(L, y)
pk   = soln[:n]

if n_active > 0:
    lk_I[Wk] = soln[-n_active:]
print(f'the new search direction pk is {pk}')
print(f'updated Lagrange multipliers {lk_I}')

the new search direction pk is [-1.75 -6.25]
updated Lagrange multipliers [0. 0. 0.]


## Note that $\lambda_i = 0$ for all inactive constraints. So after every solution of the subproblem, we set the Lagrange parameters for inactive constraints to 0. The following block of code does that

In [6]:
mask = np.full(m, True, dtype=bool)
mask[Wk] = False # identify inactive constraints
I_k = np.arange(m)[mask]
lk_I[I_k] = np.zeros_like(I_k)

# Find which constraints are blocking

In [7]:
cTp = []
for i in range(m):
    cTp.append(np.dot(Ck[i], pk))
blocking_ind = np.arange(m)[np.array(cTp) < 0]
print(f'constraints {blocking_ind + 1} are potentially blocking')

constraints [1 2 3] are potentially blocking


# Find a step length for each blocking constraint (that will make it active)

In [8]:
# step lengths
a = np.zeros(m)
for i in blocking_ind:
    a[i] = -(np.dot(Ck[i],xk.squeeze()) + d[i]) / np.dot(Ck[i], pk) 
    print(f'alpha_{i+1} = {a[i]}')
ak = min(1, min(a[blocking_ind]))
print(f'chosen step length alpha_k {ak}')

alpha_1 = 0.3595505617977528
alpha_2 = 1.7142857142857142
alpha_3 = 0.32
chosen step length alpha_k 0.32


# Now that we found appropriate step length, we take the new step

In [9]:
# take a new step
xk = xk + ak * pk
print(f'new point xk {xk}')

new point xk [[2.44 0.  ]]


# At the new step, new constraints may be active, active constraints may go inactive, so we need to update it

In [10]:
# Update working constraint set
Wk = []
for i in range(m):
    print(f'constraint {i+1} at new point {c[i](xk)}')
    if np.abs(c[i](xk)) <= 1e-12:
        Wk.append(i)
print(f'active constraints set {Wk}')        

constraint 1 at new point 0.8799999999999999
constraint 2 at new point 2.44
constraint 3 at new point 0.0
active constraints set [2]


# We have a new point, so solve the subproblem again

In [11]:
# Construct KKT system
n_active = len(Wk)
Z = np.zeros([n_active, n_active])
if n_active > 0:
    L = np.block([[Bk, -Ck[Wk].T],
                  [-Ck[Wk], Z]])
else:
    L = Bk
y = np.append(-grad(f)(xk).squeeze() + np.matmul(Ck[Wk].T, lk_I[Wk]), np.zeros(n_active) )
soln = np.linalg.solve(L, y)
pk = soln[:n]
print(soln)
lk_I[Wk] = soln[-n_active:]
print(f'the new search direction pk is {pk}')
print(f'updated Lagrange multipliers {lk_I}')

[-2.60666672e+00  6.66133815e-16  5.66666667e+00]
the new search direction pk is [-2.60666672e+00  6.66133815e-16]
updated Lagrange multipliers [0.         0.         5.66666667]


In [12]:
mask = np.full(m, True, dtype=bool)
mask[Wk] = False # identify inactive constraints
I_k = np.arange(m)[mask]
lk_I[I_k] = np.zeros_like(I_k)

## find blocking constraints

In [13]:
cTp = []
for i in range(m):
    cTp.append(np.dot(Ck[i], pk))
    print(np.dot(Ck[i], pk))
blocking_ind = np.arange(m)[np.array(cTp) < 0]
print(f'constraints {blocking_ind} are potentially blocking')

-5.21333344777425
-2.606666723887126
6.661338147750938e-16
constraints [1 2] are potentially blocking


# find step lengths for blocking constraints

In [14]:
# step lengths
a = np.zeros(m)
for i in blocking_ind:
    a[i] = -(np.dot(Ck[i],xk.squeeze()) + d[i]) / np.dot(Ck[i], pk) 
    print(f'alpha_{i+1} = {a[i]}')
ak = min(a[blocking_ind])
print(f'chosen step length alpha_k {ak}')

alpha_1 = 0.16879795025881222
alpha_2 = 0.9360613605261403
chosen step length alpha_k 0.16879795025881222


In [15]:
# take a new step
xk = xk + ak * pk
print(f'new point {xk}')

new point [[2.00000000e+00 1.12442023e-16]]


# update working set

In [16]:
# Update working constraint set
Wk = []
for i in range(m):
    print(f'constraint {i+1} at new point {c[i](xk)}')
    if np.abs(c[i](xk)) <= 1e-12:
        Wk.append(i)
print(f'active constraints set {Wk}')        

constraint 1 at new point 0.0
constraint 2 at new point 1.9999999999999998
constraint 3 at new point 1.1244202253211913e-16
active constraints set [0, 2]


# construct new subproblem

In [17]:
# Construct KKT system
n_active = len(Wk)
Z = np.zeros([n_active, n_active])
if n_active > 0:
    L = np.block([[Bk, -Ck[Wk].T],
                  [-Ck[Wk], Z]])
else:
    L = Bk
y = np.append(-grad(f)(xk).squeeze() + np.matmul(Ck[Wk].T, lk_I[Wk]), np.zeros(n_active))
soln = np.linalg.solve(L, y)
pk = soln[:n]
print(soln)
lk_I[Wk] = soln[-n_active:]
print(f'the new search direction pk is {pk}')
print(f'updated Lagrange multipliers {lk_I}')

[-3.72717730e-16  2.29974769e-16  6.50000000e+00 -1.51666665e+01]
the new search direction pk is [-3.72717730e-16  2.29974769e-16]
updated Lagrange multipliers [  6.5          0.         -15.16666651]


In [18]:
mask = np.full(m, True, dtype=bool)
mask[Wk] = False # identify inactive constraints
I_k = np.arange(m)[mask]
lk_I[I_k] = np.zeros_like(I_k)

# notice that $pk$ is 0, so now we check for signs of all Lagrange parameters

In [19]:
if np.linalg.norm(pk, np.inf) <= 1e-12: # p=0, check Lagrange parameter signs
    if any(lk_I < 0):
        print('negative Lagrange parameters, remove one of them from working set')

negative Lagrange parameters, remove one of them from working set


## When there are one or more negative Lagrange parameters, we remove the most negative constraint from the working set and retry. That is $x_{k+1} = x_k$ (so don't take the new step) and $\mathcal{W}_{k+1} = \mathcal{W}_k \backslash \{i~ \text{corresponding to most negative} \lambda_i\}$

In [20]:
negative_l_inds = np.arange(m)[lk_I < 0]

if len(lk_I[negative_l_inds]) == 0:
    print('all Lagrange parameters are nonnegative')
elif len(lk_I[negative_l_inds]) == 1:
    Wk.remove(negative_l_inds)
else:
    Wk.remove(lk_I.argmin())
print(f'updated working set {Wk}')

updated working set [0]


In [21]:
mask = np.full(m, True, dtype=bool)
mask[Wk] = False # identify inactive constraints
I_k = np.arange(m)[mask]
lk_I[I_k] = np.zeros_like(I_k)

## Now reconstruct the KKT system. Note that $x_k$ stays the same (so $B_k$, $g_k$ stay the same) however, the Lagrange parameters will be newly estimated

In [22]:
# Construct KKT system
n_active = len(Wk)
# lk_I = np.zeros(m)
Z = np.zeros([n_active, n_active])
if n_active > 0:
    L = np.block([[Bk, -Ck[Wk].T],
                  [-Ck[Wk], Z]])
else:
    L = Bk
y = np.append(-grad(f)(xk).squeeze() + np.matmul(Ck[Wk].T, lk_I[Wk]), np.zeros(n_active) )
soln = np.linalg.solve(L, y)
pk = soln[:n]
print(soln)
lk_I[Wk] = soln[-n_active:]
print(f'the new search direction is {pk}')
print(f'updated Lagrange multipliers {lk_I}')

[-1.5  1.  -3.5]
the new search direction is [-1.5  1. ]
updated Lagrange multipliers [-3.5  0.   0. ]


In [23]:
mask = np.full(m, True, dtype=bool)
mask[Wk] = False # identify inactive constraints
I_k = np.arange(m)[mask]
lk_I[I_k] = np.zeros_like(I_k)

In [24]:
cTp = []
for i in range(m):
    cTp.append(np.dot(Ck[i], pk))
    print(np.dot(Ck[i], pk))
blocking_ind = np.arange(m)[np.array(cTp) < -1e-12 ]
print(f'constraints {blocking_ind} are potentially blocking')

-8.881784197001252e-16
-1.5
0.9999999999999998
constraints [1] are potentially blocking


In [25]:
# step lengths
a = np.zeros(m)
for i in blocking_ind:
    a[i] = -(np.dot(Ck[i],xk.squeeze()) + d[i]) / np.dot(Ck[i], pk) 
    print(f'alpha_{i+1} = {a[i]}')
ak = min(1., min(a[blocking_ind]))
print(f'chosen step length alpha_k {ak}')

alpha_2 = 1.3333333333333333
chosen step length alpha_k 1.0


In [26]:
# take a new step
xk = xk + ak * pk
print(f'new point {xk}')

new point [[0.5 1. ]]


In [27]:
# Update working constraint set
Wk = []
for i in range(m):
    print(f'constraint {i+1} at new point {c[i](xk)}')
    if np.abs(c[i](xk)) <= 1e-12:
        Wk.append(i)
print(f'active constraints set {Wk}') 

constraint 1 at new point -8.881784197001252e-16
constraint 2 at new point 0.4999999999999998
constraint 3 at new point 0.9999999999999999
active constraints set [0]


In [28]:
# Construct KKT system
n_active = len(Wk)
# lk_I = np.zeros(m)
Z = np.zeros([n_active, n_active])
if n_active > 0:
    L = np.block([[Bk, -Ck[Wk].T],
                  [-Ck[Wk], Z]])
else:
    L = Bk
y = np.append(-grad(f)(xk).squeeze() + np.matmul(Ck[Wk].T, lk_I[Wk]), np.zeros(n_active) )
soln = np.linalg.solve(L, y)
pk = soln[:n]
print(soln)
lk_I[Wk] = soln[-n_active:]
print(f'the new search direction pk is {pk}')
print(f'updated Lagrange multipliers {lk_I}')

[-3.72717730e-16  2.29974769e-16  6.50000000e+00]
the new search direction pk is [-3.72717730e-16  2.29974769e-16]
updated Lagrange multipliers [6.5 0.  0. ]


# Notice that $p_k = 0$ and all Lagrange multipliers are nonnegative. So we're done! $x^* = [0.5, 1.0]^T$